__Compare FWI result with true model for the Overthrust model__

Daniel Köhn 
Kiel, 16/07/2016

__Import Libraries__

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from matplotlib.colors import LightSource, Normalize
from matplotlib.pyplot import gca
from pylab import rcParams
from matplotlib import rc
from matplotlib.ticker import FormatStrFormatter
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.ticker as mtick
import scipy.ndimage.filters
from scipy.ndimage.filters import gaussian_filter
import pickle

__Activate different post-processing options__

In [ ]:
GAUSSIAN=1;

__FD grid dimensions__

In [ ]:
DH = 0.2
NX = 500
NY = 304

__Wavefield clip value__ 

In [ ]:
clip = 5e-2
#clip = 5e1
vpmin = 60.0
vpmax = 800.0

__Define fonts__

In [ ]:
FSize = 25
font = {'color':  'black',
        'weight': 'normal',
        'size': FSize}
mpl.rc('xtick', labelsize=FSize) 
mpl.rc('ytick', labelsize=FSize) 
rcParams['figure.figsize'] = 16, 10.5

__Read FWI result and true model__

In [ ]:
#f = open("../start/Kleinneudorf_fsurf_100_smooth.vs")
f = open("../FWI_results/25_11_2017_p100_elast/modelTest_vs_stage_4.bin")
data_type = np.dtype ('float32').newbyteorder ('<')
mod_true = np.fromfile (f, dtype=data_type)
mod_true = mod_true.reshape(NX,NY)
mod_true = np.transpose(mod_true)
mod_true = np.flipud(mod_true)

In [ ]:
f = open("taper_p100.bin")
data_type = np.dtype ('float32').newbyteorder ('<')
taper = np.fromfile (f, dtype=data_type)
taper = taper.reshape(NX,NY)
taper = np.transpose(taper)
taper = np.flipud(taper)

In [ ]:
#f = open("23_11_2017_bp_40_50Hz_offset_20m_p100/jacobian_S_image.bin")
f = open("27_11_2017_bp_40_50Hz_offset_10m_p100_FWI/jacobian_S_image.bin")
data_type = np.dtype ('float32').newbyteorder ('<')
RTM_TD = np.fromfile (f, dtype=data_type)
RTM_TD = RTM_TD.reshape(NX,NY)
RTM_TD = np.transpose(RTM_TD)
RTM_TD = np.flipud(RTM_TD)
RTM_TD = scipy.ndimage.filters.laplace(RTM_TD) # suppress low-wavenumber artifacts in image
RTM_TD *= taper

__Apply Gaussian filter__

In [ ]:
if(GAUSSIAN==1):
    RTM_TD = gaussian_filter(RTM_TD, sigma=[1,6])

__Define Axis__

In [ ]:
x = np.arange(0.0, DH*NX, DH)
y = np.arange(0.0, DH*NY, DH)

__Scale RTM result with depth__

In [ ]:
RTM_scale = np.zeros((NX,NY))
RTM_scale += np.flipud(y)**4
RTM_TD*=RTM_scale.transpose()

__Define SubPlot__

In [ ]:
def do_plot(n, model, cm, an, title, vpmin, vpmax):
    
    ax=plt.subplot(1, 1, n)
    extent = [0.0,NX*DH,0.0,NY*DH]
    #plt.rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
    rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
    ## for Palatino and other serif fonts use:
    #rc('font',**{'family':'serif','serif':['Palatino']})
    #plt.rc('text', usetex=True)
    rc('text', usetex=True)
    
    if(n==1):
        im1 = plt.imshow(mod_true, cmap=plt.cm.jet, interpolation='nearest', extent=extent, aspect=1)
        plt.hold(True)

    im2 = plt.imshow(-RTM_TD, cmap=plt.cm.gray, alpha=.30, interpolation='bicubic',
                 extent=extent, vmin=-clip, vmax=clip, aspect=1)
    
    a = gca()
    a.set_xticklabels(a.get_xticks(), font)
    a.set_yticklabels(a.get_yticks(), font)
    a.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.0d'))
    a.xaxis.set_major_formatter(mtick.FormatStrFormatter('%.0d'))
    #plt.axis('scaled')
    plt.title(title, fontdict=font)
    plt.ylabel('Depth [m]', fontdict=font)
    plt.xlabel('Distance [m]', fontdict=font)
    #ax.set_xticks([]) 
    plt.gca().invert_yaxis()
    
    # add annotation
    #if n!=2:
    #    plt.text(0.5, 4.2,an,fontdict=font,color='white',size=20)
    
    # fit and label colorbar
    #divider = make_axes_locatable(ax)
    #cax = divider.append_axes("right", size="2.5%", pad=0.05)
    #cbar = plt.colorbar(im1, cax=cax)
    #cbar.set_label(an, fontdict=font, labelpad=3)

__Plot SubPlots__

In [ ]:
plt.close('all')
plt.figure()
do_plot(1, RTM_TD, 'gray', 'Vp [m/s]', r"Kleinneudorf p100 RTM result (TDFD DENISE: 40 - 50 Hz)", -clip, clip)
#do_plot(2, RTM_TD, 'gray', 'Vp [m/s]', r" ", -clip, clip)
plt.savefig('Kleinneudorf_RTM_DENISE.png', bbox_inches='tight', format='png', dpi=200)
#plt.savefig('Kleinneudorf_RTM_DENISE.pdf', bbox_inches='tight', format='pdf')
plt.tight_layout()
plt.show()